In [20]:
import cv2
import os

def extract_frames(video_path, output_folder, label, frame_interval=30):
    """
    Extract frames from a video and save them with video-level labels.

    Args:
        video_path: Path to the input video file.
        output_folder: Path to save extracted frames.
        label: Class label for the video (e.g., 1 for violence).
        frame_interval: Extract every nth frame.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Unable to open video {video_path}")
        return

    os.makedirs(output_folder, exist_ok=True)
    frame_count = 0
    saved_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            frame_name = f"{os.path.basename(video_path).split('.')[0]}_frame{frame_count}.jpg"
            frame_path = os.path.join(output_folder, frame_name)
            cv2.imwrite(frame_path, frame)
            print(f"Saved frame: {frame_path}")

            # Create pseudo-label
            label_file = frame_path.replace(".jpg", ".txt")
            with open(label_file, "w") as f:
                # YOLO format: class_id x_center y_center width height
                f.write(f"{label} 0.5 0.5 1.0 1.0\n")  # Whole frame is the label
            saved_count += 1

        frame_count += 1

    cap.release()
    print(f"Extracted {saved_count} frames from {video_path}")

# Example usage for violence and non-violence videos
dataset_path = "data"  # Root dataset folder
output_folder = "frames"  # Output folder for extracted frames

# Extract frames from violence videos
violence_folder = os.path.join(dataset_path, "violence")
for video_file in os.listdir(violence_folder):
    if video_file.endswith(('.mp4', '.avi', '.mkv')):
        extract_frames(os.path.join(violence_folder, video_file), output_folder, label=1)

# Extract frames from non-violence videos
non_violence_folder = os.path.join(dataset_path, "non_violence")
for video_file in os.listdir(non_violence_folder):
    if video_file.endswith(('.mp4', '.avi', '.mkv')):
        extract_frames(os.path.join(non_violence_folder, video_file), output_folder, label=0)


Saved frame: frames\V_986_frame0.jpg
Saved frame: frames\V_986_frame30.jpg
Saved frame: frames\V_986_frame60.jpg
Saved frame: frames\V_986_frame90.jpg
Saved frame: frames\V_986_frame120.jpg
Saved frame: frames\V_986_frame150.jpg
Extracted 6 frames from data\violence\V_986.mp4
Saved frame: frames\V_987_frame0.jpg
Saved frame: frames\V_987_frame30.jpg
Saved frame: frames\V_987_frame60.jpg
Saved frame: frames\V_987_frame90.jpg
Saved frame: frames\V_987_frame120.jpg
Extracted 5 frames from data\violence\V_987.mp4
Saved frame: frames\V_988_frame0.jpg
Saved frame: frames\V_988_frame30.jpg
Saved frame: frames\V_988_frame60.jpg
Saved frame: frames\V_988_frame90.jpg
Saved frame: frames\V_988_frame120.jpg
Extracted 5 frames from data\violence\V_988.mp4
Saved frame: frames\V_989_frame0.jpg
Saved frame: frames\V_989_frame30.jpg
Saved frame: frames\V_989_frame60.jpg
Saved frame: frames\V_989_frame90.jpg
Saved frame: frames\V_989_frame120.jpg
Extracted 5 frames from data\violence\V_989.mp4
Saved fra

In [21]:
import os
import shutil
from sklearn.model_selection import train_test_split

def organize_dataset(frames_folder):
    """
    Organize extracted frames and labels into YOLO-compatible structure.

    Args:
        frames_folder: Path to the folder containing .jpg and .txt files.
    """
    images = [f for f in os.listdir(frames_folder) if f.endswith('.jpg')]
    labels = [f for f in os.listdir(frames_folder) if f.endswith('.txt')]

    # Sort to ensure matching order
    images.sort()
    labels.sort()

    # Split dataset into train and validation sets (80% train, 20% val)
    train_images, val_images, train_labels, val_labels = train_test_split(
        images, labels, test_size=0.2, random_state=42
    )

    # Create directories
    for subdir in ['images/train', 'images/val', 'labels/train', 'labels/val']:
        os.makedirs(os.path.join(frames_folder, subdir), exist_ok=True)

    # Move files to train/val folders
    for img, lbl in zip(train_images, train_labels):
        shutil.move(
            os.path.join(frames_folder, img),
            os.path.join(frames_folder, 'images/train', img)
        )
        shutil.move(
            os.path.join(frames_folder, lbl),
            os.path.join(frames_folder, 'labels/train', lbl)
        )

    for img, lbl in zip(val_images, val_labels):
        shutil.move(
            os.path.join(frames_folder, img),
            os.path.join(frames_folder, 'images/val', img)
        )
        shutil.move(
            os.path.join(frames_folder, lbl),
            os.path.join(frames_folder, 'labels/val', lbl)
        )

    print("Dataset organized successfully!")

# Example usage
organize_dataset("frames")


Dataset organized successfully!


In [23]:
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO("yolov8n.pt")  # Using YOLOv8 Nano for faster training

# Train the model
model.train(
    data="datasets.yaml",  # Path to the dataset configuration file
    epochs=10,            # Number of training epochs
    imgsz=640,            # Image size for training
    batch=16,             # Batch size
    name="my_yolo_model"  # Name of the model run
)


Ultralytics 8.3.34  Python-3.9.11 torch-2.2.2+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=datasets.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=my_yolo_model15, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_bo

train: Scanning D:\Machine Learning\Voilencetete\frames\labels\train.cache... 255 images, 0 backgrounds, 0 corrupt: 100%|██████████| 255/255 [00:00<?, ?it/s]
val: Scanning D:\Machine Learning\Voilencetete\frames\labels\val.cache... 64 images, 0 backgrounds, 0 corrupt: 100%|██████████| 64/64 [00:00<?, ?it/s]


Plotting labels to runs\detect\my_yolo_model15\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\my_yolo_model15
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10         0G     0.6769       3.19      1.386         15        640: 100%|██████████| 16/16 [02:28<00:00,  9.30s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:12<00:00,  6.24s/it]

                   all         64         64    0.00401          1      0.957      0.861


...

                   all         64         64      0.996          1      0.995      0.995



10 epochs completed in 0.420 hours.
Optimizer stripped from runs\detect\my_yolo_model15\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\my_yolo_model15\weights\best.pt, 6.2MB

Validating runs\detect\my_yolo_model15\weights\best.pt...
Ultralytics 8.3.34  Python-3.9.11 torch-2.2.2+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:09<00:00,  4.64s/it]


                   all         64         64      0.996          1      0.995      0.995
          non_violence         57         57      0.999          1      0.995      0.995
              violence          7          7      0.992          1      0.995      0.995
Speed: 2.7ms preprocess, 122.0ms inference, 0.0ms loss, 4.2ms postprocess per image
Results saved to runs\detect\my_yolo_model15


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000231995266D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

In [31]:
from ultralytics import YOLO
import cv2

# Load the trained model
model = YOLO("D:/Machine Learning/Voilencetete/runs/detect/my_yolo_model15/weights/best.pt")  # Replace with your trained model path

def detect_violence_in_video(video_path):
    """
    Detect violence in a video and display results in real time.

    Args:
        video_path: Path to the input video file.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Unable to open video {video_path}")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Run YOLOv8 inference
        results = model(frame, conf=0.5)

        # Annotate frame
        annotated_frame = results[0].plot()
        violence_detected = False

        # Check for violence
        for r in results:
            for box in r.boxes.data:
                label = int(box[-1])  # Class ID
                if label == 1:  # Assuming 1 corresponds to 'violence'
                    violence_detected = True

        # Display annotation
        if violence_detected:
            cv2.putText(annotated_frame, "Violence Detected!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Show the video
        cv2.imshow("Violence Detection", annotated_frame)

        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Example usage
detect_violence_in_video("C:/Users/yashr/Downloads/Reordered_Video.mp4")  # Replace with your video path




0: 384x640 1 non_violence, 181.7ms
Speed: 4.0ms preprocess, 181.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 non_violence, 118.4ms
Speed: 2.0ms preprocess, 118.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 non_violence, 116.0ms
Speed: 2.0ms preprocess, 116.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 non_violence, 110.1ms
Speed: 2.0ms preprocess, 110.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 non_violence, 118.1ms
Speed: 2.4ms preprocess, 118.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 non_violence, 106.4ms
Speed: 1.0ms preprocess, 106.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 non_violence, 119.3ms
Speed: 5.3ms preprocess, 119.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 non_violence, 113.6ms
Speed: 1.8ms preprocess, 1